In [22]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class AgentState(TypedDict):
    text : str 


In [23]:
def node_a(state:AgentState):
    print("In Node A")
    return Command(
        goto="node_b",
        update={
            "text": state['text'] + "A"
        }
    )

def node_b(state:AgentState):
    print("In Node B")
    user_interrupt = interrupt("Next to Go >> ")
    print("Review Choice: ", user_interrupt)
    if user_interrupt.lower()=="c":
        
        return Command(
            goto="node_c",
            update={
                "text": state['text'] + "B"
            }
        )
    elif user_interrupt.lower()=="d":
         return Command(
            goto="node_c",
            update={
                "text": state['text'] + "B"
            }
        )
    else:
         return Command(
            goto="node_a",
            update={
                "text": state['text'] + "B"
            }
        )
    

def node_c(state:AgentState):
    print("In Node C")
    return Command(
        goto="node_d",
        update={
            "text": state['text'] + "C"
        }
    )

def node_d(state:AgentState):
    print("In Node D")
    return Command(
        goto=END,
        update={
            "text": state['text'] + "D"
        }
    )

In [24]:
graph = StateGraph(AgentState)
graph.add_node("node_a",node_a)
graph.add_node("node_b",node_b)
graph.add_node("node_c",node_c)
graph.add_node("node_d",node_d)

graph.set_entry_point("node_a")
app = graph.compile(checkpointer=memory)



In [25]:
config= {
    "configurable":{
        "thread_id":1
    }
}

result= app.invoke({"text":""}, config=config)
result

In Node A
In Node B


{'text': 'A',
 '__interrupt__': [Interrupt(value='Next to Go >> ', resumable=True, ns=['node_b:74636c00-a3d6-e109-fdee-b9ca8e0545a7'])]}

In [26]:
app.get_state(config).next

('node_b',)

In [27]:
command = Command(resume="c")
result2 = app.invoke(command,config=config)
result2

In Node B
Review Choice:  c
In Node C
In Node D


{'text': 'ABCD'}